# Коррекция ошибок

Все мы постоянно печатаем, постянно совершаем опечатки, для нас коррекция ошибок уже давно стала обыденностью. Как же она может работать? Что значит _слова похожи_? Какие неожиданные применения автокоррекция находит себе?  
Впереди нас ожидает целый шпионский триллер!


## В поисках Aшибки

Стоя перед задачей исправления ошибки в слове, сразу возникают несколько проблем:

-   Что значит, что в 'в слове ошибка'?
-   Как найти ошибку?
-   Как исправлять?
-   Из чего подбирать замену?

В ходе нашего небольшого расследования мы увидимм ответы, накопивштеся за годы в разных сферах применения. А начнём мы с простейшего подхода - словари и частотные таблицы.


In [ ]:
# Если мы не знаем слово, значит просто его нет!
# Такой девиз вполне себе применим к задаче исправления человеческого текста

russian_dictionary = ['капуста', 'крыжовник', 'картофель', 'кукуруза', 'кинза']

shoplist = """
    -Список продуктов-
    Капуста
    Куриное филе
    Куруза
    Кинза
"""

errors = [
    word
    for word in shoplist.split()
    if word.lower() not in russian_dictionary
]
print(errors)

['-Список', 'продуктов-', 'Куриное', 'филе', 'Куруза']


Нуу, существует проблема того, что размер словаря имеет значение... Зато мы засекли `куруза`!  
На удивление просто! Взять бы словарь побольше и достоверность нашего поиска ошибок увеличится. Другое дело как определять где ошибка, а главное, как их исправлять?


In [42]:
shoplist = """
    -Список продуктов-
    Капуста
    Куриное филе
    Куруза
    Кинза
"""

# Улучшим наше положение
# Зарание для коррекции ошибок разобьём текст на слова только из букв и ничего больше
# str.isalpha() возврашает истину только если обьект str состоить из букв
def tokenize(string: str):
    # побуквенно выкинем небуквы, а потом склеим и в нижний регистр
    tokens = [
        ''.join(filter(str.isalpha, word)).lower() 
        for word in string.split()
    ]
    return tokens

shoplist_tokenized = tokenize(shoplist)
print('Слова в тексте:', shoplist_tokenized)

# Словарь русского языка, содержит 1 531 464 записи
russian_dictionary = []
with open("../assets/russian.txt", mode='r') as dictionary:
    russian_dictionary = dictionary.read().split()

errors = [
        word for word in shoplist_tokenized
        if word not in russian_dictionary
    ]
print('Слова с ошибками:', *errors)

Слова в тексте: ['список', 'продуктов', 'капуста', 'куриное', 'филе', 'куруза', 'кинза']
Слова с ошибками: куруза


Так-то лучше! Ошибку мы так и не исправили, но теперь мы достоверно её отыскали!  
Недостатки? Словарь просто гигантский! (по меркам нашей небольшой программки) Он весит 30Мб!

Хорошо, раз у нас уже есть словарь, давайте выбирать замену оттуда. Как подбирать замену? Наверное надо найти самое похожее слово и подменить на него. Как измерять похожесть? Проведём исследование.


## Расстояние Хэмминга

Ричард Хэмминг - американский инжинер, программист, учёный. Прославился благодаря своему огромному вкладу в теорию помехойстойчивого кодирования. В честь него названы две вещи: Коды Хэмминга и Расстояние Хэмминга.

Мы ещё поговорим про самокорректирующие коды Хэмминга, а сейчас нас интересует Расстояние Хэмминга.


In [ ]:
# Расстояние Хэмминга - количество позиций, в которых два слова одинаковой длины отличаются
def hamming_distance(w1, w2):
    if len(w1) != len(w2): # разная длинна -> бесконечное расстояние
        return float('inf')
    return sum(a != b for a, b in zip(w1, w2))

print(hamming_distance('cat', 'bat')) # c != b - расстояние 1
print(hamming_distance('luke', 'wake'))
print(hamming_distance('крут', 'круто')) # а вот и слабое место, слова разной длинны несравнимы

1
2
inf


In [ ]:
# Вынесем нахождение ошибок в наборе нокенов в отдельную функцию
def find_errors(tokens, dictionary):
    return [
        (i, word) for i, word in enumerate(tokens) # до кучи будем запоминать позицию, в которой найдена ошибка
        if word not in dictionary
    ]

# Проверим расстояние Хэмминга - найдём ошибку и попробуем подыскать замену

errors = find_errors(shoplist_tokenized, russian_dictionary)
print(errors)

corrections = []
for i, wrong_word in errors:
    # словарь гигантский а значит, чтобы не нагружать себя обработкой всего словаря
    # обернём всё в () - получим генератор вместо списка, он сам по себе ничего не содержит
    # вместо этого он высчитывает свои элементы в момент, когда мы будем итерировать по нему
    candidates = (
        (replacement, dist) 
        for replacement, dist in 
        map(lambda w: (w, hamming_distance(w, wrong_word)), russian_dictionary)
        if dist != float('inf') # бесконечно удалённые нас не интересуют
    )
    first_10 = sorted(candidates, key=lambda a: a[1])[:10]
    print(first_10[:5], first_10[5:], sep='\n')
    

[(5, 'куруза')]
[('буруна', 2), ('бутуза', 2), ('кляуза', 2), ('комуза', 2), ('курага', 2)]
[('куража', 2), ('курдка', 2), ('курева', 2), ('курила', 2), ('курица', 2)]


Идея замечательная, имя Хэмминга не вошло бы в историю, если бы была плохой. Да вот только ошибку в слове `кукуруза` мы и близко не найдём т.к. мы потеряли буквы, а расстяние хэмминга в таком случае неприменимо. Ну давайте перед тем, как отправиться искать другой путь, всё же смоделируем более благоприятный исход и просто наделаем опечаток.

In [48]:
def suggest_corrections(located_errors, dictionary, distance_function, first_n=10):
    result = []
    for i, wrong_word in located_errors:
        candidates = (
            (replacement, dist) 
            for replacement, dist in 
            map(lambda w: (w, distance_function(w, wrong_word)), dictionary)
            if dist != float('inf') 
        )
        candidates = sorted(candidates, key=lambda a: a[1])
        result.append((
            i, 
            wrong_word,
            list(zip(*candidates))[0][:first_n]
        ))
    return result

test_string = "Я скушел многа аблок, и аблоки были очень вкусными!"
test_string_tokenized = tokenize(test_string)
test_errors = find_errors(test_string_tokenized, russian_dictionary)

In [49]:
test_suggestions = suggest_corrections(test_errors, russian_dictionary, hamming_distance)
print(*test_suggestions, sep='\n')

(1, 'скушел', ('скушал', 'скушен', 'акушер', 'вкушал', 'сдурел', 'скошен', 'скуден', 'скулеж', 'скулил', 'скупал'))
(2, 'многа', ('многи', 'много', 'множа', 'анода', 'блога', 'вноса', 'гнома', 'грога', 'дрога', 'енола'))
(3, 'аблок', ('облок', 'яблок', 'абрек', 'авлос', 'аллод', 'аплод', 'аулов', 'аулом', 'балок', 'белок'))
(5, 'аблоки', ('яблоки', 'абреки', 'волоки', 'молоки', 'облаки', 'облеки', 'облоги', 'обложи', 'обломи', 'оброки'))


Гораздо лучше! В каждом списке есть правильное предложение! Жаль, что мы замену четырёх слов искали 3.3 секунды... давайте поправим наше решение.

In [50]:
def suggest_corrections(located_errors, dictionary, distance_function, first_n=10, len_margin=4):
    result = []
    # ограничимся словами в непосредственной близости к словам в наличии
    min_len = len(min(located_errors, key=lambda a: a[1])[1]) # минимальная длина слова с ошибкой
    min_len_ = max(1, min_len - len_margin) # отрицательной длины не бывает
    max_len = len(max(located_errors, key=lambda a: a[1])[1]) + len_margin
    
    filtered_dictionary = list(filter(
        lambda word: len(word) >= min_len and len(word) <= max_len,
        dictionary
    ))
    for i, wrong_word in located_errors:
        candidates = (
            (replacement, dist) 
            for replacement, dist in 
            map(lambda w: (w, distance_function(w, wrong_word)), filtered_dictionary)
            if dist != float('inf') 
        )
        candidates = sorted(candidates, key=lambda a: a[1])
        result.append((
            i, 
            wrong_word,
            list(zip(*candidates))[0][:first_n]
        ))
    return result

test_suggestions = suggest_corrections(test_errors, russian_dictionary, hamming_distance)
print(*test_suggestions, sep='\n')

(1, 'скушел', ('скушал', 'скушен', 'акушер', 'вкушал', 'сдурел', 'скошен', 'скуден', 'скулеж', 'скулил', 'скупал'))
(2, 'многа', ('многи', 'много', 'множа', 'анода', 'блога', 'вноса', 'гнома', 'грога', 'дрога', 'енола'))
(3, 'аблок', ('облок', 'яблок', 'абрек', 'авлос', 'аллод', 'аплод', 'аулов', 'аулом', 'балок', 'белок'))
(5, 'аблоки', ('яблоки', 'абреки', 'волоки', 'молоки', 'облаки', 'облеки', 'облоги', 'обложи', 'обломи', 'оброки'))


Сэкономили секунду.   
Поехали дальше. Нам нужна другая функция расстояния, желательно чтобы она учитывала возможность удаления и вставки букв, а не только замены одних на другие. Такую уже придумали.

## Расстояние Левенштейна

Владимир Иосифович Левенштейн - советский и российский математик, доктор физико-математических наук. Ведущий научный сотрудник Института прикладной математики им. М. В. Келдыша.  

В 1965 году ввёл понятие дистанции редактирования, ныне называемой в честь него расстоянием Левенштейна. Эта метрика учитывает как и перестановки букв, так и вставки с удалениями, что делает её практически более применимой к коррекции человеческих ошибок.  

In [57]:
from functools import lru_cache

@lru_cache
def levenshtein_distance(w1, w2):
    if len(w1) == 0: # разница в длинне теперь учитывается в пользу расстояния
        return len(w2)
    if len(w2) == 0:
        return len(w1)
    if w1[0] == w2[0]: # очевидно, если начало одинаковое, то сравниваем концы
        return levenshtein_distance(w1[1:], w2[1:])
    _1 = levenshtein_distance(w1[1:], w2) # удаление буквы
    _2 = levenshtein_distance(w2[1:], w1) # вставка буквы
    _3 = levenshtein_distance(w1[1:], w2[1:]) # замена буквы
    return 1 + min(_1, _2, _3)

print(levenshtein_distance('павел', 'щавель'))

2


In [55]:
# наконец мы сможем отыскать ошибки в списке покупок
print(shoplist_tokenized)
shoplist_errors = find_errors(shoplist_tokenized, russian_dictionary)
print(shoplist_errors)
shoplist_suggestions = suggest_corrections(shoplist_errors, russian_dictionary, levenshtein_distance, len_margin=2)
print(*shoplist_suggestions, sep='\n')

['список', 'продуктов', 'капуста', 'куриное', 'филе', 'куруза', 'кинза']
[(5, 'куруза')]
(5, 'куруза', ('буруна', 'бутуза', 'картуза', 'кляуза', 'комуза', 'круиза', 'кукуруза', 'курага', 'куража', 'кургузая'))


30 секунд поиска на одно слово, и это при том, что мы кэшировали вычисления и ограничились 2 буквами разницы в длине!!  

Ну зато среди всех слов мы нашли `кукуруза` :)

In [65]:
import numpy as np

# Улучшим реализациию 
@lru_cache
def iterative_levenshtein_distance(w1, w2):
    n = len(w2)
    m = len(w1)
    v0 = np.ndarray(n + 1)
    v1 = np.ndarray(n + 1)

    for i in range(n + 1):
        v0[i] = i

    for i in range(m):
        v1[0] = i + 1
        for j in range(n):
            deletion_cost = v0[j + 1] + 1
            insertion_cost = v1[j] + 1
            substitution_cost = v0[j] if w1[i] == w2[j] else v0[j] + 1
            v1[j+1] = min(deletion_cost, insertion_cost, substitution_cost)
        v0, v1 = v1, v0

    return v0[n]
                
d = iterative_levenshtein_distance('павел', 'щавель')
print(d)

2.0


In [66]:
shoplist_suggestions = suggest_corrections(shoplist_errors, russian_dictionary, iterative_levenshtein_distance, len_margin=2)
print(*shoplist_suggestions, sep='\n')

(5, 'куруза', ('буруна', 'бутуза', 'картуза', 'кляуза', 'комуза', 'круиза', 'кукуруза', 'курага', 'куража', 'кургузая'))


Почти в три раза быстрее! Это достижение сильно улучшает применимость алгоритма. Давайте напоследок попробуем наши яблоки с расстоянием Левенштейна.

In [67]:
test_string = "Я скушел многа йаблок, и яблоки были очень вкусными!"
test_string_tokenized = tokenize(test_string)
test_errors = find_errors(test_string_tokenized, russian_dictionary)
test_suggestions = suggest_corrections(test_errors, russian_dictionary, iterative_levenshtein_distance, len_margin=2, first_n=15)
print(*test_suggestions, sep='\n')

(1, 'скушел', ('скушал', 'скушен', 'акушер', 'вкушал', 'искушал', 'искушен', 'оскудел', 'сдурел', 'скошен', 'скудели', 'скудель', 'скуден', 'скулеж', 'скулил', 'скупал'))
(2, 'многа', ('минога', 'многая', 'иногда', 'магога', 'миногам', 'миногах', 'миноге', 'миноги', 'миногу', 'минора', 'многие', 'многий', 'многим', 'многих', 'многое'))
(3, 'йаблок', ('гакблок', 'заулок', 'каблук', 'наблой', 'наблою', 'шаблон', 'арабок', 'бабенк', 'бабенок', 'бабёнок', 'бабкой', 'бабкою', 'бабник', 'бабулек', 'бабушк'))


В половине выборок мы не нашли правильной замены. Слабость расстояния Левенштейна в том, что замены весят столько же сколько и вставки и удаления, хотя доказано, что большинство совершаемых людьми ошибок - опечатки. Выходит, что мы используя более сложный алгоритм не можем найти подходящих слов.

На самом деле, для улучшения времени выполнения нам стоит:  
- Сократить словарь, ограничимся самыми частыми словами (Раза в 2-3 было бы хорошо)
- Реализовать потоковую выдачу предложений, это важный пункт для интерактивных сред
- Отсортировать словарь по частотам встречающихся слов